<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

---
title: "Core Functions"
subtitle: Essential functions for processing and visualizing the underwater cave SONAR and vision data.
authors:
  - name: Thomas Guilment
    affiliations:
      - University of Louisiana at Lafayette
    email: thomas.guilment@gmail.com
    corresponding: true
    orcid: 0009-0003-8163-3976
  - name: Gabriele Morra
    affiliations:
      - University of Louisiana at Lafayette
  - name: Orhun Aydin
    affiliations:
      - Saint Louis University
  - name: Stefany Carty
    affiliations:
      - Saint Louis University
format:
  html:
    html-math-method: katex
    self-contained: false
    page-layout: article
    number-sections: false
    toc-depth: 3
jupyter: python3
execute:
  eval: true
include: true
---

![](./img/Logo_nautilopy_tiny.png)

## Template
> Template about how to write new functions with documentation and follow coding rules. When reformatting existing functions, this template can be given to Perplexity ai (or ChatGPT).

In [1]:
#| echo: false
#| output: asis
show_doc(f_template_function)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L92){target="_blank" style="float:right; font-size:smaller"}

### f_template_function

>      f_template_function (s_param1:str, d_param2:int, m_param3:numpy.ndarray)

*Template function demonstrating coding rules, docstring format, and best practices.

::: {.callout-note}
This function serves as a template to illustrate our coding standards, documentation 
requirements, and best practices for writing high-quality, maintainable Python code.
:::

**Parameters**  
 \ \  - `s_param1`: String parameter description  
 \ \  - `d_param2`: Integer parameter description  
 \ \  - `m_param3`: numpy.ndarray parameter description  

**Returns**  
 \ \  List[float]: Description of the returned list of floats  

**Example**

```python
import numpy as np

s_example = "test"
d_value = 42
m_array = np.array([1.0, 2.0, 3.0])

result = f_template_function(s_example, d_value, m_array)
print(result)
```

::: {.callout-tip collapse=true}
#### **Coding Rules and Best Practices**

1. **Naming Conventions**:
   - Use `s_` prefix for strings
   - Use `d_` prefix for scalars (_e.g._ integers and floats)
   - Use `m_` prefix for matrixes (_e.g._ Numpy arrays)
   - Use `l_` prefix for lists
   - Use `t_` prefix for tuples
   - Use `dic_` prefix for dictionaries
   - Use `df_` prefix for dataframe
   - Use `h_` prefix for handles (e.g., file handles, plot handles)
   - Use `f_` prefix for functions
   - Use `s_` prefix for scripts
   - Use lowercase with underscores for function names

2. **Type Hinting**:
   - Always include type hints for parameters and return values

3. **Docstrings**:
   - Use nbdev-compatible docstring format
   - Include a brief description, parameters, returns, and example
   - Add a callout-tip section for detailed explanations or best practices

4. **Comments**:
   - Add detailed comments for complex operations
   - Explain the 'why' behind the code, not just the 'what'

5. **Code Structure**:
   - Keep functions focused on a single task
   - Limit function length (aim for under 50 lines)
   - Use meaningful variable names

6. **Error Handling**:
   - Use try-except blocks for error-prone operations
   - Raise custom exceptions when appropriate

7. **Performance Considerations**:
   - Use vectorized operations with numpy when possible
   - Avoid unnecessary loops or function calls

8. **Testing**:
   - Write unit tests for all functions
   - Include edge cases in your tests

9. **Version Control**:
   - Make frequent, small commits with clear messages
   - Use feature branches for new developments

10. **Documentation**:
    - Keep docstrings and comments up-to-date
    - Document any assumptions or limitations
:::

**References**  
\ \ 1. [PEP 8 -- Style Guide for Python Code](https://www.python.org/dev/peps/pep-0008/)  
\ \ 2. [NumPy Documentation](https://numpy.org/doc/stable/)*

## Fundamental Operations
> This section contains essential functions for basic AUV operations and calculations (_e.g._ converting quaternions to Euler angles, estimating 3D target position from AUV position, and SONAR pose)

In [2]:
#| echo: false
#| output: asis
show_doc(f_q2rollPitchYaw)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L201){target="_blank" style="float:right; font-size:smaller"}

### f_q2rollPitchYaw

>      f_q2rollPitchYaw (v_quaternion:numpy.ndarray)

*Convert quaternion to roll, pitch, and yaw (Euler angles) using the 3-2-1 sequence. 

::: {.callout-note}
The 3-2-1 or YZX convention is the common one used for Tait-Bryan or nautical angles.
See [wiki](https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles)
for more details. 
:::

**Parameters**  
 \ \  - `v_quaternion`: Normalized quaternion [qx, qy, qz, qw]

**Returns**  
\ \  - Tuple of (roll, pitch, yaw) angles in radians

**Example**

```python
# import numpy
import numpy as np

# Create quaternion
v_quat = np.array([0.1, 0.2, 0.3, 0.4])

# Convert quaternion to Roll, Pitch, and Yaw
d_roll, d_pitch, d_yaw = f_q2rollPitchYaw(v_quat)

# Display the results
print(f"Roll: {d_roll:.4f}, Pitch: {d_pitch:.4f}, Yaw: {d_yaw:.4f}")
```

::: {.callout-tip collapse=true}
#### **More details on quaternion to Euler Angles (Roll, Pitch, Yaw) conversion used for Tait-Bryan or nautical angles**  

Given a quaternion $q = (q_x, q_y, q_z, q_w)$, we can convert it to Euler angles (roll $\phi$, pitch $      heta$, yaw $\psi$) using the following method:

**Preliminary Calculations**  

We compute two auxiliary values:

- test = $q_z q_w + q_x q_y$
- unit = $q_x^2 + q_y^2 + q_z^2 + q_w^2$

These values are used to check for singularities and normalize the calculations.

**Singularity Handling**  
We check for singularities at the poles:

1. North Pole singularity (if test $> 0.499 \ \cdot$ unit):  
   $\psi = 2 \cdot atan2(q_x, q_w)$  
   θ = π/2  
   φ = 0  

2. South Pole singularity (if test $< -0.499 \ \cdot$ unit):  
   $\psi = -2 \cdot atan2(q_x, q_w)$  
   θ = -π/2  
   φ = 0  

**General Case**  

If no singularity is present, we compute the Euler angles as follows:  

1. Roll (φ, x-axis rotation):  
   φ = atan2(2(q_w q_x + q_y q_z), 1 - 2(q_x^2 + q_y^2))

2. Pitch (θ, y-axis rotation):  
   θ = asin(2(q_y q_w - q_z q_x)/unit)

3. Yaw ($\psi$, z-axis rotation):  
   $\psi = atan2(2(q_w q_z + q_x q_y), 1 - 2(q_y^2 + q_z^2))$

::: {.callout-note}
#### Adjustments from the references  
Note that we applied a correction from the docs here because `math.asin(2*test/unit)` is wrong (in our case)
:::

This method ensures proper handling of singularities and provides accurate conversion from quaternions to Euler angles.
:::

**References**  
\ \ 1. [Quaternion to Euler Conversion](https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles)  
\ \ 2. [Singularity Handling](https://www.euclideanspace.com/maths/geometry/rotations/conversions/quaternionToEuler/)*

In [3]:
#| echo: false
#| output: asis
show_doc(f_rollPitchYaw2q)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L369){target="_blank" style="float:right; font-size:smaller"}

### f_rollPitchYaw2q

>      f_rollPitchYaw2q (d_roll:float, d_pitch:float, d_yaw:float)

*Convert roll, pitch, and yaw angles (in radians) to a quaternion using the 3-2-1 sequence.

::: {.callout-note}
The 3-2-1 or YZX convention is the common one used for Tait-Bryan or nautical angles.
This function performs the inverse operation of [`f_q2rollPitchYaw`](https://20KUTS.github.io/nautilopy/core.html#f_q2rollpitchyaw).
:::

**Parameters**  
 \ \  - `d_roll`: The roll angle in radians  
 \ \  - `d_pitch`: The pitch angle in radians  
 \ \  - `d_yaw`: The yaw angle in radians  

**Returns**  
\ \  - Tuple representing the quaternion in the form (qx, qy, qz, qw)

**Example**

```python
# Convert Roll, Pitch, and Yaw to quaternion
d_roll, d_pitch, d_yaw = 0.1, 0.2, 0.3
v_quat = f_rollPitchYaw2q(d_roll, d_pitch, d_yaw)

# Display the results
print(f"Quaternion: ({v_quat:.4f}, {v_quat:.4f}, {v_quat:.4f}, {v_quat:.4f})")
```

::: {.callout-tip collapse=true}
#### **More details on Euler Angles (Roll, Pitch, Yaw) to quaternion conversion**

Given Euler angles (roll φ, pitch θ, yaw ψ) in radians, we can convert them to a quaternion $q = (q_x, q_y, q_z, q_w)$ using the following method:

1. Calculate intermediate values:
   - cy = cos(yaw * 0.5)
   - sy = sin(yaw * 0.5)
   - cp = cos(pitch * 0.5)
   - sp = sin(pitch * 0.5)
   - cr = cos(roll * 0.5)
   - sr = sin(roll * 0.5)

2. Compute quaternion components:
   - qw = cr * cp * cy + sr * sp * sy
   - qx = sr * cp * cy - cr * sp * sy
   - qy = cr * sp * cy + sr * cp * sy
   - qz = cr * cp * sy - sr * sp * cy

This method ensures accurate conversion from Euler angles to quaternions, maintaining the 3-2-1 (YZX) rotation sequence.
:::

**References**  
\ \ 1. [Euler Angles to Quaternion Conversion](https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles#Euler_angles_to_quaternion_conversion)  
\ \ 2. [Understanding 3D Rotations](https://www.euclideanspace.com/maths/geometry/rotations/conversions/eulerToQuaternion/index.htm)*

In [4]:
#| echo: false
#| output: asis
show_doc(f_nav2target_pos)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L475){target="_blank" style="float:right; font-size:smaller"}

### f_nav2target_pos

>      f_nav2target_pos (m_position:numpy.ndarray, m_YawPitchRoll:numpy.ndarray,
>                        m_estimated_ranges:numpy.ndarray, d_angle_rad:float)

*Calculate the target's position in global coordinates based on local sensor information.

::: {.callout-note}
This function transforms local sensor measurements into global coordinates, accounting 
for the AUV's position and orientation.
:::

**Parameters**  
 \ \  - `m_position`: Array of AUV's position (x, y, z), shape (3,) or (3, 1)  
 \ \  - `m_YawPitchRoll`: Array of yaw, pitch, roll angles in radians, shape (3,)  
 \ \  - `m_estimated_ranges`: Array of estimated ranges to the target, shape (n,) or (n, 1)  
 \ \  - `d_angle_rad`: Angle in radians representing the sensor's beam direction  

**Returns**  
\ \  - np.ndarray: Target's position in global coordinates, shape (3, n) or (3, 1)  

**Example**

```python
import numpy as np
import matplotlib.pyplot as plt

# AUV position and orientation
m_position = np.array([10, 20, -5])  # AUV at (10, 20, -5)
m_YawPitchRoll = np.array([np.pi/4, np.pi/6, 0])  # 45° yaw, 30° pitch, 0° roll

# Sensor measurements
m_estimated_ranges = np.array([50, 60, 70])  # Three range measurements
d_angle_rad = np.pi/6  # 30° sensor angle

# Calculate target positions
m_target_positions = f_nav2target_pos(m_position, m_YawPitchRoll, m_estimated_ranges, d_angle_rad)

# Visualize
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*m_position, c='r', s=100, label='AUV')
ax.scatter(m_target_positions, m_target_positions, m_target_positions, c='b', label='Targets')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
plt.title('AUV and Estimated Target Positions')
plt.show()
```

::: {.callout-tip collapse=true}
#### **Detailed explanation of the coordinate transformation process**

1. **Input Preparation**:
   - Ensure `m_position` and `m_estimated_ranges` are 2D arrays for consistent processing.

2. **Local Coordinate Calculation**:
   - Calculate x and y components in the sensor's local coordinate system using polar to Cartesian conversion:
     * x = range * cos(angle)
     * y = range * sin(angle)

3. **Rotation Matrices**:
   - Create 3D rotation matrices for yaw, pitch, and roll using Euler angles.

4. **Local to AUV Frame Transformation**:
   - Apply roll, pitch, and yaw rotations sequentially to transform from sensor frame to AUV body frame.

5. **AUV Frame to Global Frame Transformation**:
   - Add the AUV's global position to the rotated local coordinates.

This process effectively transforms the sensor's polar measurements (range and angle) 
into global Cartesian coordinates, accounting for the AUV's position and orientation in 3D space.
:::

**References**  
\ \ 1. [Coordinate Transformations in Robotics](https://cseweb.ucsd.edu/classes/wi18/cse167-a/lec3.pdf)  
\ \ 2. [SLAM Underwater Vehicle Navigation](https://ieeexplore.ieee.org/document/1638022)*

In [5]:
#| echo: false
#| output: asis
show_doc(f_hyster)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L597){target="_blank" style="float:right; font-size:smaller"}

### f_hyster

>      f_hyster (m_image:numpy.ndarray, d_low:float, d_high:float,
>                d_overlap:float=0.5, d_window_size:int=100)

*Apply hysteresis thresholding to an image with window-based processing.

::: {.callout-note}
This function performs hysteresis thresholding on an input image using a sliding window approach. 
It normalizes the image, processes it in overlapping windows, and applies local thresholding.
:::

**Parameters**  
 \ \  - `m_image`: Input image matrix  
 \ \  - `d_low`: Low threshold factor  
 \ \  - `d_high`: High threshold factor  
 \ \  - `d_overlap`: Overlap percentage between windows (default: 0.5)  
 \ \  - `d_window_size`: Size of the processing window (default: 100)  

**Returns**  
\ \  - np.ndarray: Hysteresis thresholded image matrix  

**Example**

```python
import numpy as np
import matplotlib.pyplot as plt
from skimage import data

# Load a sample image
m_image = data.camera()

# Apply hysteresis thresholding
m_result = f_hyster(m_image, d_low=0.1, d_high=0.5, d_overlap=0.5, d_window_size=100)

# Display results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.imshow(m_image, cmap='gray')
ax1.set_title('Original Image')
ax2.imshow(m_result, cmap='gray')
ax2.set_title('Hysteresis Thresholded Image')
plt.show()
```

::: {.callout-tip collapse=true}
#### **Details on the hysteresis thresholding process**

1. **Image Normalization**:
   - Normalizes the input image to the range [0, 1].

2. **Window-based Processing**:
   - Processes the image in overlapping windows.
   - Window size is determined by `d_window_size`.
   - Overlap between windows is controlled by `d_overlap`.

3. **Local Thresholding**:
   - For each window:
     a. Calculates local high and low thresholds based on the maximum intensity in the window.
     b. Applies hysteresis thresholding using `filters.apply_hysteresis_threshold`.
     c. Retains only the pixels that pass the thresholding.

4. **Result Compilation**:
   - Combines the processed windows to form the final thresholded image.

5. **Threshold Factors**:
   - `d_low` and `d_high` determine the lower and upper thresholds as fractions of the local maximum intensity.

This approach allows for adaptive thresholding that accounts for local intensity variations across the image.
:::

**References**  
\ \ 1. [Hysteresis Thresholding](https://scikit-image.org/docs/dev/auto_examples/filters/plot_hysteresis.html)  
\ \ 2. [Window-based Image Processing](https://en.wikipedia.org/wiki/Window_function)*

## Data Input/Output
> Functions for reading, writing, and managing data files.

In [6]:
#| echo: false
#| output: asis
show_doc(f_save)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L694){target="_blank" style="float:right; font-size:smaller"}

### f_save

>      f_save (s_filename:str, dic_globals:dict, *l_args:str)

*Save specified variables from the global dictionary to a pickle file.

::: {.callout-note}
This function selectively saves variables from a provided global dictionary to a file using 
pickle serialization. The saved variables can be later retrieved using the [`f_load`](https://20KUTS.github.io/nautilopy/core.html#f_load) function.
:::

**Parameters**  
 \ \  - `s_filename`: String specifying the name of the file to save the variables to  
 \ \  - `dic_globals`: Dictionary representing the global namespace containing all variables  
 \ \  - `*l_args`: Variable number of strings, each representing the name of a variable to be saved  

**Returns**  
\ \  None

**Example**

```python
import numpy as np

# Create sample variables in the global namespace
v_array = np.array([1, 2, 3, 4, 5])
dic_sample = {'a': 1, 'b': 2, 'c': 3}
s_message = "Hello, World!"

# Get the global dictionary
dic_globals = globals()

# Save the variables
f_save('my_variables.pkl', dic_globals, 'v_array', 'dic_sample', 's_message')

# These can be loaded later using:
# f_load('my_variables.pkl', globals())
```

::: {.callout-tip collapse=true}
#### **Details on the saving process**

1. **Variable Selection**:
   - Creates an empty dictionary `dic` to store selected variables.
   - Iterates through the variable names provided in `*l_args`.
   - Retrieves each variable from `dic_globals` and stores it in `dic`.

2. **File Writing**:
   - Opens the specified file (`s_filename`) in binary write mode.
   - Uses `pickle.dump()` to serialize the `dic` dictionary and write it to the file.

3. **Serialization**:
   - Employs pickle for serialization, allowing saving of complex Python objects.
   - The resulting file is in binary format and not human-readable.

4. **Scope Consideration**:
   - Only saves variables present in the provided `dic_globals` dictionary.
   - Allows for more flexible use of the function in different contexts.

5. **File Naming**:
   - The filename should include the desired file extension (e.g., '.pkl' for pickle files).
   - Ensure write permissions in the target directory.
:::

**References**  
\ \ 1. [Python Pickle Module](https://docs.python.org/3/library/pickle.html)  
\ \ 2. [Python Global and Local Scope](https://docs.python.org/3/tutorial/classes.html#python-scopes-and-namespaces)*

In [7]:
#| echo: false
#| output: asis
show_doc(f_load)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L770){target="_blank" style="float:right; font-size:smaller"}

### f_load

>      f_load (s_filename:str, dic_globals:dict)

*Load variables from a pickle file and update the provided global dictionary.

::: {.callout-note}
This function retrieves variables saved by [`f_save`](https://20KUTS.github.io/nautilopy/core.html#f_save) from a pickle file and updates the 
provided global dictionary with these variables. It uses the pickle module for deserialization.
:::

**Parameters**  
 \ \  - `s_filename`: String specifying the name of the file to load variables from  
 \ \  - `dic_globals`: Dictionary representing the global namespace to update with loaded variables  

**Returns**  
\ \  None

**Example**

```python
# Assume we previously saved variables using:
# f_save('my_variables.pkl', globals(), 'v_array', 'dic_sample', 's_message')

# To load these variables:
dic_globals = globals()
f_load('my_variables.pkl', dic_globals)

# Now v_array, dic_sample, and s_message are available in the global namespace
print(v_array)
print(dic_sample)
print(s_message)
```

::: {.callout-tip collapse=true}
#### **Details on the loading process**

1. **File Reading**:
   - Opens the specified file (`s_filename`) in binary read mode.
   - Uses `pickle.load()` to deserialize the content of the file into a dictionary.

2. **Variable Restoration**:
   - The loaded dictionary contains variable names as keys and their values.
   - Uses the `update()` method of the provided global dictionary (`dic_globals`) 
     to add or update variables with the loaded values.

3. **Deserialization**:
   - Pickle is used for deserialization, allowing restoration of complex Python objects.

4. **Scope Consideration**:
   - The function updates the provided dictionary, which is expected to be the global namespace.
   - This approach allows for more flexible use of the function in different contexts.

5. **Security Note**:
   - Pickle can execute arbitrary code during deserialization.
   - Only load files from trusted sources to avoid potential security risks.
:::

**References**  
\ \ 1. [Python Pickle Module](https://docs.python.org/3/library/pickle.html)  
\ \ 2. [Python Global and Local Scope](https://docs.python.org/3/tutorial/classes.html#python-scopes-and-namespaces)*

In [8]:
#| echo: false
#| output: asis
show_doc(f_generate_unique_filename)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L837){target="_blank" style="float:right; font-size:smaller"}

### f_generate_unique_filename

>      f_generate_unique_filename (s_directory:str)

*Generate a unique filename that does not already exist in the current directory.

::: {.callout-note}
This function checks if a given filename exists in the current directory. If it does, 
it appends a counter to the base filename until a unique filename is found.
:::

**Parameters**  
 \ \  - `s_directory`: String representing the filename (without the path) to use as the base for the new filename  

**Returns**  
\ \  - String representing the new unique filename  

**Example**

```python
import os

# Create a test file
with open('test_file.txt', 'w') as f:
    f.write('This is a test file')

# Generate a unique filename
s_new_filename = f_generate_unique_filename('test_file.txt')
print(f"Generated unique filename: {s_new_filename}")

# Try again with the same base filename
s_another_filename = f_generate_unique_filename('test_file.txt')
print(f"Generated another unique filename: {s_another_filename}")

# Clean up
os.remove('test_file.txt')
```

::: {.callout-tip collapse=true}
#### **Details on the unique filename generation process**

1. **Existence Check**:
   - Checks if the given filename already exists in the current directory.

2. **Filename Parsing**:
   - If the file exists, splits the filename into base name and extension.

3. **Counter Initialization**:
   - Initializes a counter to 1 for appending to the base filename.

4. **Unique Name Generation**:
   - Enters a loop to generate new filenames by appending the counter to the base name.
   - Checks if each generated filename exists in the current directory.
   - Increments the counter if the generated filename already exists.

5. **Return Value**:
   - Returns the original filename if it doesn't exist in the directory.
   - Returns the first generated filename that doesn't exist in the directory.

This approach ensures that the function always returns a filename that can be safely 
used without overwriting existing files.
:::

**References**  
\ \ 1. [Python os.path Module](https://docs.python.org/3/library/os.path.html)  
\ \ 2. [File and Directory Operations in Python](https://docs.python.org/3/library/os.html#files-and-directories)*

In [9]:
#| echo: false
#| output: asis
show_doc(f_save_var)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L918){target="_blank" style="float:right; font-size:smaller"}

### f_save_var

>      f_save_var (s_directory:str, dic_globals:dict, *l_args:str)

*Save specified variables from the global dictionary to individual pickle files in a unique directory.

::: {.callout-note}
This function creates a unique directory and saves each specified variable as a separate pickle file 
within that directory. It uses the [`f_generate_unique_filename`](https://20KUTS.github.io/nautilopy/core.html#f_generate_unique_filename) function to ensure a unique directory name.
:::

**Parameters**  
 \ \  - `s_directory`: String specifying the base name for the directory to save the files in  
 \ \  - `dic_globals`: Dictionary representing the global namespace containing all variables  
 \ \  - `*l_args`: Variable number of strings, each representing the name of a variable to be saved  

**Returns**  
\ \  None

**Example**

```python
import numpy as np
import os
import shutil

# Create sample variables in the global namespace
v_array = np.array([1, 2, 3, 4, 5])
dic_sample = {'a': 1, 'b': 2, 'c': 3}
s_message = "Hello, World!"

# Get the global dictionary
dic_globals = globals()

# Save the variables
f_save_var('saved_variables', dic_globals, 'v_array', 'dic_sample', 's_message')

# Check the contents of the created directory
s_created_dir = [d for d in os.listdir() if d.startswith('saved_variables')]
print(f"Created directory: {s_created_dir}")
print("Saved files:")
for s_file in os.listdir(s_created_dir):
    print(f" - {s_file}")

# Clean up (remove the created directory)
shutil.rmtree(s_created_dir)
```

::: {.callout-tip collapse=true}
#### **Details on the variable saving process**

1. **Directory Creation**:
   - Uses [`f_generate_unique_filename`](https://20KUTS.github.io/nautilopy/core.html#f_generate_unique_filename) to create a unique directory name.
   - Creates the directory using `os.makedirs` with `exist_ok=True` to avoid errors if the directory already exists.

2. **Variable Saving**:
   - Iterates through each variable name provided in `*l_args`.
   - For each variable:
     a. Constructs a filename by joining the directory path and the variable name with a '.pkl' extension.
     b. Opens the file in binary write mode.
     c. Uses `pickle.dump()` to serialize the variable and write it to the file.

3. **File Naming**:
   - Each variable is saved in a separate file named after the variable with a '.pkl' extension.

4. **Scope Consideration**:
   - Only saves variables present in the provided `dic_globals` dictionary.

5. **Error Handling**:
   - The function will raise an error if a specified variable is not found in `dic_globals`.

This approach allows for organized storage of multiple variables, each in its own file within a unique directory.
:::

**References**  
\ \ 1. [Python Pickle Module](https://docs.python.org/3/library/pickle.html)  
\ \ 2. [Python os Module](https://docs.python.org/3/library/os.html)  
\ \ 3. [Python File I/O](https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files)*

In [10]:
#| echo: false
#| output: asis
show_doc(f_load_var)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L1008){target="_blank" style="float:right; font-size:smaller"}

### f_load_var

>      f_load_var (s_path:str, d_glob:Dict, l_var:Optional[List[str]]=None,
>                  b_verbose=False)

*Load variables from pickle files into the global namespace.

::: {.callout-note}
This function loads variables from pickle files located in the specified path.
If a list of variable names is provided, it loads only those variables.
If no list is provided, it loads all pickle files in the directory.
:::

**Parameters**  
 \ \  - `s_path`: String path to the directory containing pickle files  
 \ \  - `d_glob`: Dictionary representing the global namespace (usually globals())  
 \ \  - `l_var`: Optional list of variable names to load. If None, loads all pickle files.  

**Returns**  
 \ \  None

**Example**

```python
import os

# Load specific variables
l_var = ['m_pos_auv', 'm_ypr']
glob = globals()
f_load_var("./data/Pickle_dataset/", glob, l_var)
del glob

# Load all pickle files in the directory
glob = globals()
f_load_var("./data/Pickle_dataset/", glob)
del glob
```

::: {.callout-tip collapse=true}
#### **Details on variable loading process**

1. If `l_var` is not provided, the function will load all .pkl files in the directory.
2. For each variable to be loaded:
   - Constructs the full file path.
   - Checks if the file exists.
   - If it exists, loads the pickle file and assigns it to the global namespace.
   - If it doesn't exist, prints a warning message.
3. The function modifies the global namespace directly through the `d_glob` parameter.
:::

**References**  
\ \ 1. [Python pickle module](https://docs.python.org/3/library/pickle.html)  
\ \ 2. [Python globals() function](https://docs.python.org/3/library/functions.html#globals)*

## Data Analysis and Processing
> Functions for data exploration, analysis, and transformation.

In [11]:
#| echo: false
#| output: asis
show_doc(f_pointCloud)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L1078){target="_blank" style="float:right; font-size:smaller"}

### f_pointCloud

>      f_pointCloud (m_pos:numpy.ndarray, m_ypr:numpy.ndarray,
>                    m_sonar:numpy.ndarray, v_angle:numpy.ndarray,
>                    d_max_range:float, d_thresh:float=0,
>                    b_return_intensity:bool=False)

*Create a 3D point cloud from sensor data, typically used in underwater sonar applications.

::: {.callout-note}
This function processes sonar data to generate a 3D point cloud. It handles sensor position, orientation, 
and intensity thresholding.
:::

**Parameters**  
 \ \  - `m_pos`: Array of sensor positions (x, y, z), shape (n, 3)  
 \ \  - `m_ypr`: Array of sensor orientations (yaw, pitch, roll) in radians, shape (n, 3)  
 \ \  - `m_sonar`: Array of sonar intensity data, shape (m, n) where m is the number of range bins  
 \ \  - `v_angle`: Array of sonar beam angles in radians, shape (n,)  
 \ \  - `d_max_range`: Maximum range of the sonar sensor  
 \ \  - `d_thresh`: Intensity threshold for point inclusion (default: 0)  
 \ \  - `b_return_intensity`: Flag to return intensity values (default: False)  

**Returns**  
\ \  - If `b_return_intensity` is False: np.ndarray of shape (k, 3) representing the 3D point cloud  
\ \  - If `b_return_intensity` is True: Tuple (point_cloud, intensities), where intensities is a list of arrays  

**Example**

```python
import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt

# Generate sample data
n_samples = 100
m_pos = np.random.rand(n_samples, 3) * 10
m_ypr = np.random.rand(n_samples, 3) * np.pi
m_sonar = np.random.rand(50, n_samples) * 100  # 50 range bins
v_angle = np.linspace(-np.pi/4, np.pi/4, n_samples)
d_max_range = 50.0

# Generate point cloud
point_cloud = f_pointCloud(m_pos, m_ypr, m_sonar, v_angle, d_max_range, d_thresh=50)

# Visualize point cloud
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(point_cloud[:, 0], point_cloud[:, 1], point_cloud[:, 2], c='b', marker='.')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Point Cloud from Sonar Data')
plt.show()
```

::: {.callout-tip collapse=true}
#### **Detailed explanation of the point cloud generation process**

1. **Data Preparation**:
   - Invert the Z-axis of positions to match underwater conventions.
   - Generate a range array based on the maximum range and sonar data shape.
   - Apply a 180° roll rotation to the orientation data for correct alignment.

2. **Intensity Thresholding**:
   - If `d_thresh` is -1, only the maximum intensity points are kept.
   - Otherwise, points with intensity above `d_thresh` are retained.

3. **Point Cloud Generation**:
   - For each sonar measurement:
     a. Calculate target positions using the [`f_nav2target_pos`](https://20KUTS.github.io/nautilopy/core.html#f_nav2target_pos) function (not shown here).
     b. This function likely converts polar coordinates (range, angle) to Cartesian coordinates.
   - Concatenate all calculated positions into a single point cloud array.

4. **Intensity Handling**:
   - If `b_return_intensity` is True, the function also returns the intensity values for each point.

This process effectively transforms raw sonar data into a 3D spatial representation, 
accounting for sensor movement and orientation changes during data collection.
:::

**References**  
\ \ None*

## Visualization
> All functions related to plotting, animating, and creating visual data representations.

In [12]:
#| echo: false
#| output: asis
show_doc(f_plot_rotated_axes)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L1195){target="_blank" style="float:right; font-size:smaller"}

### f_plot_rotated_axes

>      f_plot_rotated_axes (h_ax:matplotlib.axes._axes.Axes,
>                           r_rotation:scipy.spatial.transform._rotation.Rotatio
>                           n, s_name:Optional[str]=None,
>                           t_offset:Tuple[float,float,float]=(0, 0, 0),
>                           d_scale:float=1)

*Plot rotated coordinate axes on a 3D plot based on a given rotation.

::: {.callout-note}
This function visualizes the orientation of a rotated coordinate system. It helps visualize
3D rotations and transformations.
:::

**Parameters**  
 \ \  - `h_ax`: The Matplotlib 3D axes object where the rotated axes will be plotted  
 \ \  - `r_rotation`: A scipy.spatial.transform.Rotation object representing the rotation  
 \ \  - `s_name`: Name for the axes (optional)  
 \ \  - `t_offset`: A 3-element tuple specifying the offset for the axes (default is (0, 0, 0))  
 \ \  - `d_scale`: Scale factor for the length of the axes (default is 1)  

**Returns**  
\ \  None

**Example**

```python
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R

# Create a 3D plot
fig = plt.figure()
h_ax = fig.add_subplot(111, projection='3d')

# Create a rotation (45 degrees around x, 30 around y, and 60 around z)
r_rotation = R.from_euler('xyz', [45, 30, 60], degrees=True)

# Plot the rotated axes
f_plot_rotated_axes(h_ax, r_rotation, s_name='Rotated Axes', d_scale=2)

# Set plot limits and labels
h_ax.set_xlim(-3, 3)
h_ax.set_ylim(-3, 3)
h_ax.set_zlim(-3, 3)
h_ax.set_xlabel('X')
h_ax.set_ylabel('Y')
h_ax.set_zlabel('Z')

plt.show()
```

::: {.callout-tip collapse=true}
#### **More details on plotting rotated axes**

The function performs the following steps to plot the rotated axes:

1. Define colorblind-safe RGB colors for x, y, and z axes.
2. For each axis (x, y, z):
   a. Create a line representing the axis.
   b. Apply the rotation to the line.
   c. Plot the rotated line.
   d. Add a text label for the axis.
3. If a name is provided, add a text label at the origin of the rotated axes.

The rotation is applied using the `apply` method of the `Rotation` object, which
transforms the coordinates of the axis lines and text labels.

The `d_scale` parameter allows adjusting the length of the plotted axes, which can be
useful for visualizing rotations at different scales or in crowded plots.
:::

**References**  
\ \ 1. [Matplotlib 3D plotting](https://matplotlib.org/stable/tutorials/toolkits/mplot3d.html)  
\ \ 2. [SciPy Spatial Transform Rotation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html)*

In [13]:
#| echo: false
#| output: asis
show_doc(f_plot_3d_trajectory)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L1294){target="_blank" style="float:right; font-size:smaller"}

### f_plot_3d_trajectory

>      f_plot_3d_trajectory (ax:matplotlib.axes._axes.Axes,
>                            m_trajectory:numpy.ndarray, s_color:str='purple')

*Plot a 3D trajectory on the specified axes, highlighting start and endpoints.

**Parameters**  
 \ \  - `ax`: Matplotlib axes object for 3D plotting  
 \ \  - `m_trajectory`: 3D trajectory points array of shape (n, 3), where n is the number of points  
 \ \  - `s_color`: Color of the trajectory (default: 'purple')  

**Returns**  
\ \  None

**Example**

```python
import numpy as np
import matplotlib.pyplot as plt

# Create a sample 3D trajectory
t = np.linspace(0, 10, 100)
x = t * np.cos(t)
y = t * np.sin(t)
z = t
m_trajectory = np.column_stack((x, y, z))

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the trajectory
f_plot_3d_trajectory(ax, m_trajectory, s_color='blue')

# Set labels and title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Trajectory Visualization')

plt.show()
```

::: {.callout-tip collapse=true}
#### **More details on 3D trajectory plotting**

The function performs the following steps to visualize the 3D trajectory:

1. Plot the entire trajectory:
   - Uses `ax.plot()` to draw a continuous line representing the trajectory.
   - The color of the line is specified by the `s_color` parameter.

2. Mark the start point:
   - Uses `ax.scatter()` to place a green marker at the first point of the trajectory.
   - This helps identify where the trajectory begins.

3. Mark the end point:
   - Uses `ax.scatter()` to place a red marker at the last point of the trajectory.
   - This clearly shows where the trajectory ends.

4. Add a legend:
   - Calls `ax.legend()` to add a legend to the plot, explaining the color coding.

The function assumes that the input `m_trajectory` is a NumPy array where each row represents 
a point in 3D space (x, y, z coordinates). The function is flexible and can handle trajectories 
of any length.
:::

**References**  
\ \ 1. [Matplotlib 3D plotting](https://matplotlib.org/stable/tutorials/toolkits/mplot3d.html)  
\ \ 2. [NumPy array manipulation](https://numpy.org/doc/stable/user/basics.indexing.html)*

In [14]:
#| echo: false
#| output: asis
show_doc(f_image_grid)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L1377){target="_blank" style="float:right; font-size:smaller"}

### f_image_grid

>      f_image_grid (l_imgs_path:List[str], d_max_nrow:int=4, d_max_ncol:int=5)

*Load and display images in a grid layout from a list of image file paths.

::: {.callout-note}
This function creates a visual grid of images, useful for comparing multiple images 
or creating image catalogs. It automatically resizes images and adjusts the layout 
for optimal viewing.
:::

**Parameters**  
 \ \  - `l_imgs_path`: List of strings, each representing a path to an image file  
 \ \  - `d_max_nrow`: Integer specifying the maximum number of rows in the grid (default: 4)  
 \ \  - `d_max_ncol`: Integer specifying the maximum number of columns in the grid (default: 5)  

**Returns**  
\ \  None

**Example**

```python
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os

# Create some sample images
def create_sample_image(s_filename, d_color):
    m_img = np.full((256, 512, 3), d_color, dtype=np.uint8)
    s_img = Image.fromarray(m_img)
    s_img.save(s_filename)
    return s_filename

l_sample_images = [
    create_sample_image('red.png', (255, 0, 0)),
    create_sample_image('green.png', (0, 255, 0)),
    create_sample_image('blue.png', (0, 0, 255)),
    create_sample_image('yellow.png', (255, 255, 0)),
    create_sample_image('cyan.png', (0, 255, 255)),
    create_sample_image('magenta.png', (255, 0, 255))
]

# Display the images in a grid
f_image_grid(l_sample_images, d_max_nrow=2, d_max_ncol=3)
plt.show()

# Clean up sample images
for s_file in l_sample_images:
    os.remove(s_file)
```

::: {.callout-tip collapse=true}
#### **Details on the image grid creation process**

1. **Figure Creation**:
   - Creates a figure with a size of (30, 30/1.618), using the golden ratio for aesthetics.

2. **Image Processing**:
   - Iterates through each image path in the provided list.
   - Opens each image using PIL (Python Imaging Library).
   - Resizes each image to a fixed size of 512x256 pixels for consistency.

3. **Grid Layout**:
   - Uses matplotlib's subplot functionality to create a grid layout.
   - Adjusts the layout to minimize space between images (wspace and hspace).

4. **Image Display**:
   - Creates a new subplot for each image.
   - Turns off the axis for each subplot to focus on the image content.
   - Displays the resized image in its respective subplot.

5. **Flexibility**:
   - The function adapts to different numbers of images and grid sizes.
   - If there are fewer images than grid cells, empty cells will be left blank.

This approach provides a flexible and visually appealing way to display multiple images 
in a grid format, suitable for various image analysis and presentation tasks.
:::

**References**  
\ \ 1. [Matplotlib Subplots](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html)  
\ \ 2. [PIL (Python Imaging Library)](https://pillow.readthedocs.io/en/stable/)  
\ \ 3. [Image Resizing Techniques](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.resize)*

In [15]:
#| echo: false
#| output: asis
show_doc(f_plot_auv_orientation_subplots)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L1491){target="_blank" style="float:right; font-size:smaller"}

### f_plot_auv_orientation_subplots

>      f_plot_auv_orientation_subplots (m_pos_auv:numpy.ndarray,
>                                       m_ypr:numpy.ndarray)

*Plot AUV orientation with different view angles in a 2x2 subplot layout.

::: {.callout-note}
This function visualizes the AUV's trajectory and orientation from four different 
perspectives, providing a comprehensive view of the AUV's movement and rotation in 3D space.
:::

**Parameters**  
 \ \  - `m_pos_auv`: numpy.ndarray containing AUV position data (x, y, z coordinates)  
 \ \  - `m_ypr`: numpy.ndarray containing yaw, pitch, roll data for AUV orientation  

**Returns**  
\ \  None

**Example**

```python
import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt

# Generate sample data
d_num_points = 2200
m_pos_auv = np.column_stack((
    np.linspace(0, 100, d_num_points),  # x
    10 * np.sin(np.linspace(0, 4*np.pi, d_num_points)),  # y
    -5 * np.cos(np.linspace(0, 2*np.pi, d_num_points))   # z
))
m_ypr = np.column_stack((
    np.linspace(0, 2*np.pi, d_num_points),  # yaw
    0.2 * np.sin(np.linspace(0, 4*np.pi, d_num_points)),  # pitch
    0.1 * np.cos(np.linspace(0, 6*np.pi, d_num_points))   # roll
))

# Plot AUV orientation
f_plot_auv_orientation_subplots(m_pos_auv, m_ypr)
```

::: {.callout-tip collapse=true}
#### **Details on the AUV orientation plotting process**

1. **Data Preparation**:
   - Select a subset of the input data (indices 9000 to 11100) for visualization.
   - Inverts the z-axis values to match the conventional "depth" representation.

2. **Figure Setup**:
   - Creates a figure with a size of 12x12 inches.

3. **Subplot Creation**:
   - Creates four 3D subplots in a 2x2 grid, each with a different view angle:
     a. (30, -45): Perspective view
     b. (0, 0): Front view
     c. (0, 90): Side view
     d. (90, 90): Top view

4. **AUV Orientation Visualization**:
   - For each subplot:
     a. Plots rotated axes at regular intervals along the trajectory using [`f_plot_rotated_axes`](https://20KUTS.github.io/nautilopy/core.html#f_plot_rotated_axes).
     b. The axes represent the AUV's orientation (yaw, pitch, roll) at each point.

5. **Trajectory Plotting**:
   - Plots the AUV's trajectory using [`f_plot_3d_trajectory`](https://20KUTS.github.io/nautilopy/core.html#f_plot_3d_trajectory).

6. **View Adjustment**:
   - Sets the aspect ratio to "auto" for each subplot.
   - Adjusts the view angle for each subplot to provide different perspectives.

7. **Layout Adjustment**:
   - Adjusts the spacing between subplots to maximize the use of figure space.

This approach provides a comprehensive visualization of the AUV's movement and orientation, 
allowing for analysis from multiple angles simultaneously.
:::

**References**  
\ \ 1. [Matplotlib 3D Plotting](https://matplotlib.org/stable/tutorials/toolkits/mplot3d.html)  
\ \ 2. [SciPy Rotation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html)*

In [16]:
#| echo: false
#| output: asis
show_doc(f_plot_auv_orientation_trajectory)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L1610){target="_blank" style="float:right; font-size:smaller"}

### f_plot_auv_orientation_trajectory

>      f_plot_auv_orientation_trajectory (m_pos_auv:numpy.ndarray,
>                                         m_ypr:numpy.ndarray)

*Plot AUV orientation over trajectory using interactive widgets.

::: {.callout-note}
This function creates an interactive 3D plot showing the AUV's trajectory and its orientation 
at different points along the path. It uses ipywidgets to allow dynamic exploration of the AUV's 
orientation throughout its journey.
:::

**Parameters**  
 \ \  - `m_pos_auv`: numpy.ndarray containing AUV position data (x, y, z coordinates)  
 \ \  - `m_ypr`: numpy.ndarray containing yaw, pitch, roll data for AUV orientation  

**Returns**  
\ \  None

**Example**

```python
import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# Generate sample data
d_num_points = 1000
m_pos_auv = np.column_stack((
    np.linspace(0, 100, d_num_points),  # x
    10 * np.sin(np.linspace(0, 4*np.pi, d_num_points)),  # y
    -5 * np.cos(np.linspace(0, 2*np.pi, d_num_points))   # z
))
m_ypr = np.column_stack((
    np.linspace(0, 2*np.pi, d_num_points),  # yaw
    0.2 * np.sin(np.linspace(0, 4*np.pi, d_num_points)),  # pitch
    0.1 * np.cos(np.linspace(0, 6*np.pi, d_num_points))   # roll
))

# Plot interactive AUV orientation
f_plot_auv_orientation_trajectory(m_pos_auv, m_ypr)
```

::: {.callout-tip collapse=true}
#### **Details on the interactive AUV orientation plotting process**

1. **Data Preparation**:
   - Uses the entire input data range for visualization.
   - Inverts the z-axis values to represent depth.

2. **Figure Setup**:
   - Creates a figure with a size of 12x9 inches and 100 DPI.
   - Sets up a 3D subplot with orthographic projection and custom aspect ratio.

3. **Trajectory Plotting**:
   - Plots the entire AUV trajectory.
   - Marks the start (green) and end (red) points of the trajectory.

4. **Orientation Data Preparation**:
   - Pre-computes rotation objects for each data point to improve performance.

5. **Interactive Visualization**:
   - Uses `ipywidgets.interact` to create an interactive slider.
   - The slider controls which point along the trajectory is currently visualized.

6. **Update Function**:
   - `update_plot` function is called whenever the slider value changes.
   - It updates the AUV orientation axes at the selected point on the trajectory.
   - Removes previous orientation axes and redraws them at the new position.

7. **Plot Customization**:
   - Sets axis labels, title, and legend.
   - Adjusts plot limits to ensure the entire trajectory is visible.

This approach provides an interactive way to explore the AUV's orientation changes 
along its trajectory, allowing for detailed analysis of its movement and rotation.
:::

**References**  
\ \ 1. [Matplotlib 3D Plotting](https://matplotlib.org/stable/tutorials/toolkits/mplot3d.html)  
\ \ 2. [ipywidgets Documentation](https://ipywidgets.readthedocs.io/en/latest/)*

In [17]:
#| echo: false
#| output: asis
show_doc(f_create_auv_orientation_animation_plotly)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L1743){target="_blank" style="float:right; font-size:smaller"}

### f_create_auv_orientation_animation_plotly

>      f_create_auv_orientation_animation_plotly (m_pos_auv:numpy.ndarray,
>                                                 m_ypr:numpy.ndarray,
>                                                 v_timestamp_nav:numpy.ndarray)

*Create a 3D trajectory animation with animated reference axes.

::: {.callout-note}
This function generates a Plotly figure containing a 3D animation of an AUV's trajectory,
including reference axes that update based on the AUV's orientation.
:::

**Parameters**  
 \ \  - `m_pos_auv`: numpy.ndarray containing AUV positions (N x 3)  
 \ \  - `m_ypr`: numpy.ndarray containing yaw, pitch, roll values (N x 3)  
 \ \  - `v_timestamp_nav`: numpy.ndarray containing timestamps (N,)  

**Returns**  
 \ \  go.Figure: Plotly figure object containing the 3D trajectory animation  

**Example**

```python
import numpy as np

m_pos_auv = np.random.rand(100, 3) * 100  # Random positions
m_ypr = np.random.rand(100, 3) * np.pi  # Random orientations
v_timestamp_nav = np.linspace(0, 1000, 100)  # Timestamps

h_fig = f_create_auv_orientation_animation_plotly(m_pos_auv, m_ypr, v_timestamp_nav)
h_fig.show()
```

::: {.callout-tip collapse=true}
#### **Implementation Details**

1. The function first processes the input data to create a trajectory.
2. It then adds the trajectory, start point, and end point to the figure.
3. A bounding box is calculated to ensure all points are visible.
4. The initial position and axes are added to the figure.
5. Animation frames are created for each step of the trajectory.
6. The layout is updated with animation controls and time slider.

Performance considerations:
- Use of numpy operations for efficient data processing.
- Plotly's Frame objects for smooth animations.
- Adjustable step size for balancing animation smoothness and performance.
:::

**References**  
\ \ 1. [Plotly Python Documentation](https://plotly.com/python/)  
\ \ 2. [SciPy Spatial Transform Documentation](https://docs.scipy.org/doc/scipy/reference/spatial.transform.html)*

In [18]:
#| echo: false
#| output: asis
show_doc(f_add_trajectory_traces)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L1808){target="_blank" style="float:right; font-size:smaller"}

### f_add_trajectory_traces

>      f_add_trajectory_traces (h_fig:plotly.graph_objs._figure.Figure,
>                               m_trajectory:numpy.ndarray)

*Add trajectory, start, and end point traces to the figure.

::: {.callout-note}
This function adds three traces to the given figure: the full trajectory,
the start point, and the end point.
:::

**Parameters**  
 \ \  - `h_fig`: Plotly figure object to which traces will be added  
 \ \  - `m_trajectory`: numpy.ndarray containing trajectory points (N x 3)  

**Returns**  
 \ \  None  

**Example**

```python
import plotly.graph_objects as go
import numpy as np

h_fig = go.Figure()
m_trajectory = np.random.rand(100, 3)
f_add_trajectory_traces(h_fig, m_trajectory)
h_fig.show()
```*

In [19]:
#| echo: false
#| output: asis
show_doc(f_create_axes)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L1850){target="_blank" style="float:right; font-size:smaller"}

### f_create_axes

>      f_create_axes (t_position:Tuple[float,float,float],
>                     r_rotation:scipy.spatial.transform._rotation.Rotation,
>                     d_scale:float=4)

*Create 3D axes at a given position and orientation.

::: {.callout-note}
This function generates three 3D scatter traces representing the X, Y, and Z axes
at a specified position and orientation.
:::

**Parameters**  
 \ \  - `t_position`: Tuple of (x, y, z) coordinates for the axes origin  
 \ \  - `r_rotation`: Rotation object specifying the orientation of the axes  
 \ \  - `d_scale`: Scale factor for the axes length (default: 4)  

**Returns**  
 \ \  List[go.Scatter3d]: List of three Scatter3d objects representing the axes  

**Example**

```python
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

t_position = (0, 0, 0)
r_rotation = R.from_euler('xyz', [0, 0, 45], degrees=True)
l_axes = f_create_axes(t_position, r_rotation)

h_fig = go.Figure(data=l_axes)
h_fig.show()
```*

In [20]:
#| echo: false
#| output: asis
show_doc(f_add_bounding_box)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L1898){target="_blank" style="float:right; font-size:smaller"}

### f_add_bounding_box

>      f_add_bounding_box (h_fig:plotly.graph_objs._figure.Figure,
>                          m_trajectory:numpy.ndarray)

*Calculate and add bounding box to the figure.

::: {.callout-note}
This function calculates the bounding box for the given trajectory and
updates the figure's layout to ensure all points are visible.
:::

**Parameters**  
 \ \  - `h_fig`: Plotly figure object to be updated  
 \ \  - `m_trajectory`: numpy.ndarray containing trajectory points (N x 3)  

**Returns**  
 \ \  None  

**Example**

```python
import plotly.graph_objects as go
import numpy as np

h_fig = go.Figure()
m_trajectory = np.random.rand(100, 3) * 100
f_add_bounding_box(h_fig, m_trajectory)
h_fig.show()
```*

In [21]:
#| echo: false
#| output: asis
show_doc(f_add_initial_position_and_axes)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L1941){target="_blank" style="float:right; font-size:smaller"}

### f_add_initial_position_and_axes

>      f_add_initial_position_and_axes (h_fig:plotly.graph_objs._figure.Figure,
>                                       m_trajectory:numpy.ndarray,
>                                       m_ypr:numpy.ndarray)

*Add initial position and axes to the figure.

::: {.callout-note}
This function adds a marker for the initial position and 3D axes representing
the initial orientation to the figure.
:::

**Parameters**  
 \ \  - `h_fig`: Plotly figure object to be updated  
 \ \  - `m_trajectory`: numpy.ndarray containing trajectory points (N x 3)  
 \ \  - `m_ypr`: numpy.ndarray containing yaw, pitch, roll values (N x 3)  

**Returns**  
 \ \  None  

**Example**

```python
import plotly.graph_objects as go
import numpy as np

h_fig = go.Figure()
m_trajectory = np.random.rand(100, 3) * 100
m_ypr = np.random.rand(100, 3) * np.pi
f_add_initial_position_and_axes(h_fig, m_trajectory, m_ypr)
h_fig.show()
```*

In [22]:
#| echo: false
#| output: asis
show_doc(f_add_animation_frames)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L1979){target="_blank" style="float:right; font-size:smaller"}

### f_add_animation_frames

>      f_add_animation_frames (h_fig:plotly.graph_objs._figure.Figure,
>                              m_trajectory:numpy.ndarray, m_ypr:numpy.ndarray)

*Create and add animation frames to the figure.

::: {.callout-note}
This function generates animation frames for the trajectory, updating the
position and orientation of the AUV at each step.
:::

**Parameters**  
 \ \  - `h_fig`: Plotly figure object to be updated  
 \ \  - `m_trajectory`: numpy.ndarray containing trajectory points (N x 3)  
 \ \  - `m_ypr`: numpy.ndarray containing yaw, pitch, roll values (N x 3)  

**Returns**  
 \ \  None  

**Example**

```python
import plotly.graph_objects as go
import numpy as np

h_fig = go.Figure()
m_trajectory = np.random.rand(100, 3) * 100
m_ypr = np.random.rand(100, 3) * np.pi
f_add_animation_frames(h_fig, m_trajectory, m_ypr)
h_fig.show()
```*

In [23]:
#| echo: false
#| output: asis
show_doc(f_update_layout)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L2022){target="_blank" style="float:right; font-size:smaller"}

### f_update_layout

>      f_update_layout (h_fig:plotly.graph_objs._figure.Figure,
>                       m_trajectory:numpy.ndarray,
>                       v_timestamp_nav:numpy.ndarray)

*Update the layout of the figure with animation controls.

::: {.callout-note}
This function adds animation controls and a time slider to the figure layout.
:::

**Parameters**  
 \ \  - `h_fig`: Plotly figure object to be updated  
 \ \  - `m_trajectory`: numpy.ndarray containing trajectory points (N x 3)  
 \ \  - `v_timestamp_nav`: numpy.ndarray containing timestamps for each point  

**Returns**  
 \ \  None  

**Example**

```python
import plotly.graph_objects as go
import numpy as np

h_fig = go.Figure()
m_trajectory = np.random.rand(100, 3) * 100
v_timestamp_nav = np.linspace(1609459200, 1609462800, 100)  # 1 hour of timestamps
f_update_layout(h_fig, m_trajectory, v_timestamp_nav)
h_fig.show()
```*

In [ ]:
def f_add_trajectory_traces(h_fig: go.Figure, m_trajectory: np.ndarray) -> None:
    """Add trajectory, start, and end point traces to the figure."""
    h_fig.add_trace(go.Scatter3d(
        x=m_trajectory[:, 0], y=m_trajectory[:, 1], z=m_trajectory[:, 2],
        mode='lines', name='Trajectory', line=dict(color='purple', width=2)
    ))
    h_fig.add_trace(go.Scatter3d(
        x=[m_trajectory[0, 0]], y=[m_trajectory[0, 1]], z=[m_trajectory[0, 2]],
        mode='markers', name='Start', marker=dict(color='green', size=2)
    ))
    h_fig.add_trace(go.Scatter3d(
        x=[m_trajectory[-1, 0]], y=[m_trajectory[-1, 1]], z=[m_trajectory[-1, 2]],
        mode='markers', name='End', marker=dict(color='red', size=2)
    ))

def f_create_axes(t_position: Tuple[float, float, float], r_rotation: R, d_scale: float = 4) -> List[go.Scatter3d]:
    """Create 3D axes at a given position and orientation."""
    m_axes = np.array([[d_scale, 0, 0], [0, d_scale, 0], [0, 0, d_scale]])
    m_rotated_axes = r_rotation.apply(m_axes)
    l_colors = ['red', 'green', 'blue']
    l_labels = ['X', 'Y', 'Z']
    
    return [go.Scatter3d(
        x=[t_position[0], t_position[0] + m_rotated_axes[d_i, 0]],
        y=[t_position[1], t_position[1] + m_rotated_axes[d_i, 1]],
        z=[t_position[2], t_position[2] + m_rotated_axes[d_i, 2]],
        mode='lines+text',
        line=dict(color=l_colors[d_i], width=5),
        text=['', l_labels[d_i]],
        textposition='top center',
        showlegend=False
    ) for d_i in range(3)]

def f_add_bounding_box(h_fig: go.Figure, m_trajectory: np.ndarray) -> None:
    """Calculate and add bounding box to the figure."""
    d_x_min, d_x_max = np.min(m_trajectory[:, 0]), np.max(m_trajectory[:, 0])
    d_y_min, d_y_max = np.min(m_trajectory[:, 1]), np.max(m_trajectory[:, 1])
    d_z_min, d_z_max = np.min(m_trajectory[:, 2]), np.max(m_trajectory[:, 2])

    d_padding = 0.1 * max(d_x_max - d_x_min, d_y_max - d_y_min, d_z_max - d_z_min)
    h_fig.update_layout(scene=dict(
        xaxis=dict(range=[d_x_min - d_padding, d_x_max + d_padding], title='X'),
        yaxis=dict(range=[d_y_min - d_padding, d_y_max + d_padding], title='Y'),
        zaxis=dict(range=[d_z_min - d_padding, d_z_max + d_padding], title='Z'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.5),
        camera=dict(eye=dict(x=1.15, y=1.15, z=1), up=dict(x=0, y=0, z=1))
    ))

def f_add_initial_position_and_axes(h_fig: go.Figure, m_trajectory: np.ndarray, m_ypr: np.ndarray) -> None:
    """Add initial position and axes to the figure."""
    r_initial_rotation = R.from_euler('ZYX', m_ypr[0] + np.array([0, 0, np.pi]))
    h_fig.add_trace(go.Scatter3d(
        x=[m_trajectory[0, 0]], y=[m_trajectory[0, 1]], z=[m_trajectory[0, 2]],
        mode='markers', marker=dict(color='blue', size=4), name='Current Position'
    ))
    h_fig.add_traces(f_create_axes(tuple(m_trajectory[0]), r_initial_rotation))

def f_add_animation_frames(h_fig: go.Figure, m_trajectory: np.ndarray, m_ypr: np.ndarray) -> None:
    """Create and add animation frames to the figure."""
    d_step = 100
    l_frames = []
    for d_i in range(0, len(m_trajectory), d_step):
        l_frame_data = [go.Scatter3d(
            x=[m_trajectory[d_i, 0]], y=[m_trajectory[d_i, 1]], z=[m_trajectory[d_i, 2]],
            mode='markers', marker=dict(color='blue', size=4), showlegend=False
        )]
        r_rotation = R.from_euler('ZYX', m_ypr[d_i] + np.array([0, 0, np.pi]))
        l_frame_data.extend(f_create_axes(tuple(m_trajectory[d_i]), r_rotation))
        l_frames.append(go.Frame(data=l_frame_data, traces=[3, 4, 5, 6], name=f'frame{d_i}'))
    h_fig.frames = l_frames

def f_format_time(d_timestamp: float) -> str:
    """Convert timestamp to HH:MM:SS format."""
    return datetime.utcfromtimestamp(d_timestamp).strftime('%H:%M:%S')

def f_update_layout(h_fig: go.Figure, m_trajectory: np.ndarray, v_timestamp_nav: np.ndarray) -> None:
    """Update the layout of the figure with animation controls."""
    d_step = 100
    h_fig.update_layout(
        title='3D Trajectory with Animated Reference Axes',
        width=800,
        height=600,
        updatemenus=[dict(
            type='buttons',
            showactive=False,
            buttons=[
                dict(label='Play', method='animate', args=[None, dict(frame=dict(duration=50, redraw=True), fromcurrent=True, mode='immediate')]),
                dict(label='Pause', method='animate', args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')])
            ]
        )],
        sliders=[dict(
            active=0,
            yanchor='top',
            xanchor='left',
            currentvalue=dict(font=dict(size=16), prefix='UTC Time: ', visible=True, xanchor='right'),
            transition=dict(duration=300, easing='cubic-in-out'),
            pad=dict(b=10, t=50),
            len=0.9,
            x=0.1,
            y=0,
            steps=[dict(
                method='animate',
                label=f_format_time(v_timestamp_nav[d_i]),
                args=[[f'frame{d_i}'], dict(frame=dict(duration=50, redraw=True), mode='immediate', transition=dict(duration=0))]
            ) for d_i in range(0, len(m_trajectory), d_step)]
        )]
    )

# Usage example
# m_pos_auv = np.random.rand(100, 3) * 100  # Random positions
# m_ypr = np.random.rand(100, 3) * np.pi  # Random orientations
# v_timestamp_nav = np.linspace(0, 1000, 100)  # Timestamps
# h_fig = f_create_auv_orientation_animation_plotly(m_pos_auv, m_ypr, v_timestamp_nav)
# h_fig.show()

In [24]:
#| echo: false
#| output: asis
show_doc(f_create_auv_orientation_animation)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L2083){target="_blank" style="float:right; font-size:smaller"}

### f_create_auv_orientation_animation

>      f_create_auv_orientation_animation (m_pos_auv:numpy.ndarray,
>                                          m_ypr:numpy.ndarray)

*Create an animation of AUV orientation over its trajectory.

::: {.callout-note}
This function generates a dynamic visualization of an AUV's movement and orientation 
changes along its trajectory. It creates a Matplotlib animation that can be saved 
or displayed in various formats, including interactive notebooks.
:::

**Parameters**  
 \ \  - `m_pos_auv`: numpy.ndarray containing AUV position data (x, y, z coordinates)  
 \ \  - `m_ypr`: numpy.ndarray containing yaw, pitch, roll data for AUV orientation  

**Returns**  
 \ \  - matplotlib.animation.FuncAnimation: Animation object of the AUV's trajectory and orientation  

**Example**

```python
import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Generate sample data
d_num_points = 1000
m_pos_auv = np.column_stack((
    np.linspace(0, 100, d_num_points),  # x
    10 * np.sin(np.linspace(0, 4*np.pi, d_num_points)),  # y
    -5 * np.cos(np.linspace(0, 2*np.pi, d_num_points))   # z
))
m_ypr = np.column_stack((
    np.linspace(0, 2*np.pi, d_num_points),  # yaw
    0.2 * np.sin(np.linspace(0, 4*np.pi, d_num_points)),  # pitch
    0.1 * np.cos(np.linspace(0, 6*np.pi, d_num_points))   # roll
))

# Create animation
anim = f_create_auv_orientation_animation(m_pos_auv, m_ypr)

# Display animation in notebook
HTML(anim.to_jshtml())

# Alternatively, save animation to file
# anim.save('auv_orientation.gif', writer='pillow', fps=30)
```

::: {.callout-tip collapse=true}
#### **Details on the AUV orientation animation process**

1. **Data Preparation**:
   - Uses the entire input data range for animation.
   - Inverts the z-axis values to represent depth.
   - Samples data at regular intervals (every 200 points by default) to reduce computation.

2. **Figure Setup**:
   - Creates a figure with a size of 12x9 inches and 100 DPI.
   - Sets up a 3D subplot with orthographic projection and custom aspect ratio.

3. **Trajectory Plotting**:
   - Plots the entire AUV trajectory.
   - Marks the start (green) and end (red) points of the trajectory.

4. **Orientation Data Preparation**:
   - Pre-computes rotation objects for each sampled data point to improve performance.

5. **Animation Function**:
   - Defines `animate` function that updates the plot for each frame:
     a. Removes previous AUV orientation axes.
     b. Plots new orientation axes at the current position.
     c. Redraws the trajectory and start/end points.
     d. Updates axis limits and labels.

6. **Animation Creation**:
   - Uses `matplotlib.animation.FuncAnimation` to create the animation.
   - Sets the interval between frames and the total number of frames.

7. **Memory Management**:
   - Closes the figure after creating the animation to prevent displaying the initial plot and save memory.

This approach creates a smooth animation of the AUV's movement and orientation changes, 
providing a dynamic visualization of its 3D trajectory and rotational behavior.
:::

**References**  
\ \ 1. [Matplotlib Animation](https://matplotlib.org/stable/api/animation_api.html)*

In [25]:
#| echo: false
#| output: asis
show_doc(f_plot_polar_sonar_seaking)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L2224){target="_blank" style="float:right; font-size:smaller"}

### f_plot_polar_sonar_seaking

>      f_plot_polar_sonar_seaking (m_sonar:numpy.ndarray, v_angle:numpy.ndarray)

*Plot the polar view of sonar data from a SeaKing sonar or similar device.

::: {.callout-note}
This function creates a polar plot of sonar data, typically used for visualizing 
underwater acoustic measurements. It helps analyze the 
spatial distribution of sonar intensities around the sensor.
:::

**Parameters**  
 \ \  - `m_sonar`: numpy.ndarray of sonar intensity data, shape (n_ranges, n_angles)  
 \ \  - `v_angle`: numpy.ndarray of angle data in radians, shape (n_angles,)  

**Returns**  
 \ \  None

**Example**

```python
import numpy as np
import matplotlib.pyplot as plt

# Generate sample sonar data
d_n_ranges = 100
d_n_angles = 360
m_sonar = np.random.rand(d_n_ranges, d_n_angles)
v_angle = np.linspace(0, 2*np.pi, d_n_angles)

# Create some artificial features in the sonar data
m_sonar[30:40, 180:200] = 0.8  # Simulated object
m_sonar[60:70, 90:110] = 0.9   # Another simulated object

# Plot the sonar data
f_plot_polar_sonar_seaking(m_sonar, v_angle)
plt.show()
```

::: {.callout-tip collapse=true}
#### **Details on the polar sonar plot creation**

1. **Figure Setup**:
   - Creates a figure with a size of 10x10 inches.
   - Uses a polar projection for the subplot.

2. **Data Plotting**:
   - Uses `pcolormesh` to create a color-mapped representation of the sonar data.
   - The radial dimension represents range, and the angular dimension represents angle.

3. **Polar Plot Customization**:
   - Sets the angle direction to clockwise.
   - Offsets the angle by π/2 to place 0° at the top.

4. **Tick Label Adjustment**:
   - Customizes the angular tick labels to show degrees from 0° to 330°.

5. **Axis Labeling**:
   - Adds labels for angle and range axes.
   - Adjusts label padding for better visibility.

6. **Colorbar and Title**:
   - Adds a colorbar to represent sonar intensity.
   - Sets a title for the plot and labels for the axes.

This visualization approach allows for intuitive interpretation of sonar data, 
making it easier to identify objects or features in the scanned area.
:::

**References**  
\ \ 1. [Matplotlib Polar Plots](https://matplotlib.org/stable/gallery/pie_and_polar_charts/polar_demo.html)*

In [26]:
#| echo: false
#| output: asis
show_doc(f_plot_polar_sonar_micron)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L2324){target="_blank" style="float:right; font-size:smaller"}

### f_plot_polar_sonar_micron

>      f_plot_polar_sonar_micron (m_sonar:numpy.ndarray, v_angle:numpy.ndarray)

*Plot the polar view of sonar data from a Micron sonar or similar horizontal scanning device.

::: {.callout-note}
This function creates a polar plot of horizontal sonar data, typically used for visualizing 
underwater acoustic measurements in a planar view. It's particularly useful for analyzing 
the spatial distribution of sonar intensities in a horizontal plane around the sensor.
:::

**Parameters**  
 \ \  - `m_sonar`: numpy.ndarray of sonar intensity data, shape (n_ranges, n_angles)  
 \ \  - `v_angle`: numpy.ndarray of angle data in radians, shape (n_angles,)  

**Returns**  
 \ \  None

**Example**

```python
import numpy as np
import matplotlib.pyplot as plt

# Generate sample horizontal sonar data
d_n_ranges = 200  # Increased range for horizontal sonar
d_n_angles = 360
m_sonar = np.random.rand(d_n_ranges, d_n_angles)
v_angle = np.linspace(0, 2*np.pi, d_n_angles)

# Create some artificial features in the sonar data
m_sonar[50:70, 180:200] = 0.8  # Simulated distant object
m_sonar[100:120, 90:110] = 0.9   # Another simulated object
m_sonar[30:40, 270:290] = 0.7  # Closer object

# Plot the sonar data
f_plot_polar_sonar_micron(m_sonar, v_angle)
plt.show()
```

::: {.callout-tip collapse=true}
#### **Details on the horizontal polar sonar plot creation**

1. **Figure Setup**:
   - Creates a figure with a size of 10x10 inches.
   - Uses a polar projection for the subplot, suitable for horizontal scanning.

2. **Data Plotting**:
   - Uses `pcolormesh` to create a color-mapped representation of the sonar data.
   - The radial dimension represents range (up to 20 meters), and the angular dimension represents angle.

3. **Polar Plot Customization**:
   - Sets the angle direction to clockwise, matching typical horizontal sonar conventions.
   - Offsets the angle by π/2 to place 0° at the top, aligning with standard navigation orientations.

4. **Tick Label Adjustment**:
   - Customizes the angular tick labels to show degrees from 0° to 330°, covering a full 360° scan.

5. **Axis Labeling**:
   - Adds labels for angle and range axes.
   - Adjusts label padding for better visibility in the circular plot.

6. **Colorbar and Title**:
   - Adds a colorbar to represent sonar intensity.
   - Sets a title specifying "Horizontal Sonar" and labels the range axis up to 20 meters.

This visualization approach is tailored for horizontal scanning sonars, providing an 
intuitive top-down view of the underwater environment, which is crucial for navigation, 
obstacle detection, and mapping in autonomous underwater vehicles (AUVs) or remotely 
operated vehicles (ROVs).
:::

**References**  
\ \ 1. [Micron Sonar Documentation](https://www.tritech.co.uk/product/micron-sonar)*

In [27]:
#| echo: false
#| output: asis
show_doc(f_plot_horizontal_sonar)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L2427){target="_blank" style="float:right; font-size:smaller"}

### f_plot_horizontal_sonar

>      f_plot_horizontal_sonar (m_sonar:numpy.ndarray, d_start_micron:int,
>                               d_image_width:int, v_range_micron:numpy.ndarray)

*Plot the horizontal sonar data in a 2D image format.

::: {.callout-note}
This function creates a 2D visualization of horizontal sonar data, typically used for 
displaying continuous sonar scans over time. It's particularly useful for analyzing 
temporal changes in the underwater environment and detecting objects or features 
across multiple sonar revolutions.
:::

**Parameters**  
 \ \  - `m_sonar`: numpy.ndarray of sonar intensity data, shape (n_ranges, n_time_steps)  
 \ \  - `d_start_micron`: Integer representing the starting index of the sonar data  
 \ \  - `d_image_width`: Integer specifying the width of the sonar image in time steps  
 \ \  - `v_range_micron`: numpy.ndarray representing the range values of the sonar data  

**Returns**  
 \ \  None

**Example**

```python
import numpy as np
import matplotlib.pyplot as plt

# Generate sample horizontal sonar data
d_n_ranges = 397  # 397 bins over 20 meters
d_n_time_steps = 2000  # Multiple revolutions
m_sonar = np.random.rand(d_n_ranges, d_n_time_steps)
v_range_micron = np.linspace(0, 20, d_n_ranges)  # 20 meters range

# Create some artificial features in the sonar data
m_sonar[50:70, 400:600] = 0.8  # Simulated object across multiple scans
m_sonar[100:120, 800:1000] = 0.9  # Another simulated object
m_sonar[30:40, 1200:1400] = 0.7  # Closer object

# Plot the sonar data
f_plot_horizontal_sonar(m_sonar, d_start_micron=194, d_image_width=1400, v_range_micron=v_range_micron)
plt.show()
```

::: {.callout-tip collapse=true}
#### **Details on the horizontal sonar plot creation**

1. **Figure Setup**:
   - Creates a figure with a size of 16x9 inches, suitable for widescreen displays.

2. **Data Plotting**:
   - Uses `imshow` to create a 2D color-mapped representation of the sonar data.
   - The y-axis represents range (up to 20 meters), and the x-axis represents time or scan number.

3. **Plot Customization**:
   - Sets a title "Horizontal Sonar" with large font size.
   - Labels the x-axis to indicate the time scale (1 revolution every 200 points).
   - Labels the y-axis to show the range scale (397 bins over 20 meters).
   - Adjusts tick label sizes for better readability.

4. **Image Extent**:
   - Sets the extent of the image to match the start index and image width.
   - Inverts the y-axis so that closer ranges appear at the top of the plot.

5. **Colorbar Addition**:
   - Adds a colorbar to the right of the main plot to represent sonar intensity.
   - Adjusts the colorbar position and size to fit well with the main plot.

This visualization approach is particularly useful for analyzing continuous sonar data, 
allowing for the detection of persistent features, moving objects, or temporal changes 
in the underwater environment. It provides a comprehensive view of the sonar data across 
multiple scans or revolutions.
:::

**References**  
\ \ None*

In [28]:
#| echo: false
#| output: asis
show_doc(f_plot_horizontal_sonar_with_angles)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L2524){target="_blank" style="float:right; font-size:smaller"}

### f_plot_horizontal_sonar_with_angles

>      f_plot_horizontal_sonar_with_angles (m_sonar:numpy.ndarray,
>                                           v_angles_rad:numpy.ndarray,
>                                           d_start_micron:int,
>                                           d_image_width:int,
>                                           v_range_micron:numpy.ndarray)

*Plot the horizontal sonar data in a 2D image format with angle information.

::: {.callout-note}
This function creates a 2D visualization of horizontal sonar data, incorporating angle 
information on the x-axis. It's particularly useful for analyzing the angular distribution 
of sonar intensities across a full 360-degree scan, allowing for precise localization of 
underwater features or objects.
:::

**Parameters**  
 \ \  - `m_sonar`: numpy.ndarray of sonar intensity data, shape (n_ranges, n_angles)  
 \ \  - `v_angles_rad`: numpy.ndarray of angle data in radians  
 \ \  - `d_start_micron`: Integer representing the starting index of the sonar data  
 \ \  - `d_image_width`: Integer specifying the width of the sonar image in angle steps  
 \ \  - `v_range_micron`: numpy.ndarray representing the range values of the sonar data  

**Returns**  
 \ \  None

**Example**

```python
import numpy as np
import matplotlib.pyplot as plt

# Generate sample horizontal sonar data
d_n_ranges = 397  # 397 bins over 20 meters
d_n_angles = 200  # One full revolution
m_sonar = np.random.rand(d_n_ranges, d_n_angles)
v_range_micron = np.linspace(0, 20, d_n_ranges)  # 20 meters range
v_angles_rad = np.linspace(0, 2*np.pi, d_n_angles)

# Create some artificial features in the sonar data
m_sonar[50:70, 30:50] = 0.8  # Simulated object at about 45 degrees
m_sonar[100:120, 90:110] = 0.9  # Another object at about 180 degrees
m_sonar[30:40, 150:170] = 0.7  # Closer object at about 270 degrees

# Plot the sonar data
f_plot_horizontal_sonar_with_angles(m_sonar, v_angles_rad, d_start_micron=0, d_image_width=200, v_range_micron=v_range_micron)
plt.show()
```

::: {.callout-tip collapse=true}
#### **Details on the horizontal sonar plot creation with angles**

1. **Figure Setup**:
   - Creates a figure with a size of 16x9 inches, suitable for widescreen displays.

2. **Data Plotting**:
   - Uses `imshow` to create a 2D color-mapped representation of the sonar data.
   - The y-axis represents range (up to 20 meters), and the x-axis represents angles (0 to 360 degrees).

3. **Angle Conversion**:
   - Converts the input angles from radians to degrees for more intuitive visualization.

4. **Plot Customization**:
   - Sets a title "Horizontal Sonar" with large font size.
   - Labels the x-axis to indicate the angular scale (360 degrees per revolution).
   - Labels the y-axis to show the range scale (397 bins over 20 meters).
   - Adjusts tick label sizes for better readability.

5. **Angular Tick Adjustment**:
   - Sets custom tick labels for angles, showing degrees from 0° to 360° at 30° intervals.

6. **Grid Addition**:
   - Adds a grid to the plot for easier reference of angles and ranges.

7. **Colorbar Addition**:
   - Adds a colorbar to the right of the main plot to represent sonar intensity.
   - Adjusts the colorbar position and size to fit well with the main plot.

This visualization approach provides a comprehensive view of the sonar data across a full 
360-degree scan, allowing for precise angular localization of underwater features or objects. 
It's particularly useful for navigation, obstacle detection, and environmental mapping in 
autonomous underwater vehicles (AUVs) or remotely operated vehicles (ROVs).
:::

**References**  
\ \ None*

In [29]:
#| echo: false
#| output: asis
show_doc(f_plot_vertical_sonar_with_angles)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L2640){target="_blank" style="float:right; font-size:smaller"}

### f_plot_vertical_sonar_with_angles

>      f_plot_vertical_sonar_with_angles (m_sonar:numpy.ndarray,
>                                         v_angles_rad:numpy.ndarray,
>                                         d_start:int, d_image_width:int,
>                                         v_range:numpy.ndarray)

*Plot the vertical sonar data in a 2D image format with angle information.

::: {.callout-note}
This function creates a 2D visualization of vertical sonar data, incorporating angle 
information on the x-axis. It's particularly useful for analyzing the vertical profile 
of underwater environments, allowing for the detection of features such as seafloor 
topography, water column structures, or suspended objects across a full 360-degree scan.
:::

**Parameters**  
 \ \  - `m_sonar`: numpy.ndarray of sonar intensity data, shape (n_ranges, n_angles)  
 \ \  - `v_angles_rad`: numpy.ndarray of angle data in radians  
 \ \  - `d_start`: Integer representing the starting index of the sonar data  
 \ \  - `d_image_width`: Integer specifying the width of the sonar image in angle steps  
 \ \  - `v_range`: numpy.ndarray representing the range values of the sonar data  

**Returns**  
 \ \  None

**Example**

```python
import numpy as np
import matplotlib.pyplot as plt

# Generate sample vertical sonar data
d_n_ranges = 50  # 50 bins over 10 meters
d_n_angles = 201  # One full revolution
m_sonar = np.random.rand(d_n_ranges, d_n_angles)
v_range = np.linspace(0, 10, d_n_ranges)  # 10 meters range
v_angles_rad = np.linspace(0, 2*np.pi, d_n_angles)

# Create some artificial features in the sonar data
m_sonar[20:30, 30:50] = 0.8  # Simulated seafloor feature at about 45 degrees
m_sonar[10:15, 90:110] = 0.9  # Water column structure at about 180 degrees
m_sonar[5:10, 150:170] = 0.7  # Near-surface object at about 270 degrees

# Plot the sonar data
f_plot_vertical_sonar_with_angles(m_sonar, v_angles_rad, d_start=0, d_image_width=201, v_range=v_range)
plt.show()
```

::: {.callout-tip collapse=true}
#### **Details on the vertical sonar plot creation with angles**

1. **Figure Setup**:
   - Creates a figure with a size of 16x9 inches, suitable for widescreen displays.

2. **Data Plotting**:
   - Uses `imshow` to create a 2D color-mapped representation of the sonar data.
   - The y-axis represents range (up to 10 meters), and the x-axis represents angles (0 to 360 degrees).

3. **Angle Conversion**:
   - Converts the input angles from radians to degrees for more intuitive visualization.

4. **Plot Customization**:
   - Sets a title "Vertical Sonar" with large font size.
   - Labels the x-axis to indicate the angular scale (360 degrees per revolution, 201 points).
   - Labels the y-axis to show the range scale (50 bins over 10 meters).
   - Adjusts tick label sizes for better readability.

5. **Angular Tick Adjustment**:
   - Sets custom tick labels for angles, showing degrees from 0° to 360° at 30° intervals.

6. **Grid Addition**:
   - Adds a grid to the plot for easier reference of angles and ranges.

7. **Colorbar Addition**:
   - Adds a colorbar to the right of the main plot to represent sonar intensity.
   - Adjusts the colorbar position and size to fit well with the main plot.

This visualization approach provides a comprehensive view of the vertical sonar data across 
a full 360-degree scan, allowing for analysis of water column structures, seafloor profiling, 
and detection of suspended objects. It's particularly useful for environmental monitoring, 
bathymetry studies, and vertical obstacle detection in underwater robotics applications.
:::

**References**  
\ \ None*

In [30]:
#| echo: false
#| output: asis
show_doc(f_animation_AUV_micron_trajectory)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L2756){target="_blank" style="float:right; font-size:smaller"}

### f_animation_AUV_micron_trajectory

>      f_animation_AUV_micron_trajectory
>                                         (m_interp_pos_micron_final:numpy.ndarr
>                                         ay, m_sonarReceivedIntensity_micron_fi
>                                         nal:numpy.ndarray,
>                                         v_range_micron:numpy.ndarray)

*Create an animation of AUV trajectory along with horizontal sonar data visualization.

::: {.callout-note}
This function generates a dynamic visualization combining the AUV's trajectory in 3D space 
with its horizontal sonar data. It provides a comprehensive view of the AUV's movement and 
the underwater environment it's scanning, useful for mission analysis and data interpretation.
:::

**Parameters**  
 \ \  - `m_interp_pos_micron_final`: numpy.ndarray of interpolated AUV position data, shape (n_points, 3)  
 \ \  - `m_sonarReceivedIntensity_micron_final`: numpy.ndarray of sonar intensity data, shape (n_ranges, n_points)  
 \ \  - `v_range_micron`: numpy.ndarray representing the range values of the sonar data  

**Returns**  
 \ \  matplotlib.animation.FuncAnimation: Animation object of the AUV trajectory and sonar data  

**Example**

```python
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# Generate sample data
d_n_points = 50000
d_n_ranges = 397

# AUV trajectory data
m_interp_pos_micron_final = np.column_stack((
    np.linspace(0, 100, d_n_points),  # x
    10 * np.sin(np.linspace(0, 8*np.pi, d_n_points)),  # y
    -5 * np.cos(np.linspace(0, 4*np.pi, d_n_points))   # z
))

# Sonar intensity data
m_sonarReceivedIntensity_micron_final = np.random.rand(d_n_ranges, d_n_points)
v_range_micron = np.linspace(0, 20, d_n_ranges)

# Create animation
anim = f_animation_AUV_micron_trajectory(m_interp_pos_micron_final, 
                                         m_sonarReceivedIntensity_micron_final, 
                                         v_range_micron)

# Display animation in notebook
HTML(anim.to_jshtml())
```

::: {.callout-tip collapse=true}
#### **Details on the AUV trajectory and sonar animation process**

1. **Animation Setup**:
   - Creates a figure with a 16:9 aspect ratio, suitable for widescreen displays.
   - Loads and resizes a background image for context (e.g., satellite view of the mission area).

2. **Update Function**:
   - Defines `update_plot` function that is called for each frame of the animation.
   - Clears the previous frame and sets up a new grid of subplots for each update.

3. **Trajectory Visualization**:
   - Plots the full AUV trajectory and the current segment in two 3D subplots.
   - One subplot shows a top-down view, the other a perspective view.
   - Uses color-coding for start and end points, and different colors for full and current trajectories.

4. **Sonar Data Visualization**:
   - Displays the horizontal sonar data as a 2D color plot below the trajectory plots.
   - Shows a moving window of sonar data corresponding to the current position in the trajectory.

5. **Layout and Styling**:
   - Uses a grid layout to organize the background image, 3D trajectory plots, and sonar data plot.
   - Applies consistent styling, including font sizes and color schemes.

6. **Animation Creation**:
   - Uses `matplotlib.animation.FuncAnimation` to create the animation.
   - Sets the frame range, interval, and links it to the update function.

This approach provides a dynamic and comprehensive visualization of both the AUV's movement 
through 3D space and the sonar data it collects along its trajectory, enabling intuitive 
interpretation of the mission data and environmental features.
:::

**References**  
\ \ 1. [Matplotlib Animation](https://matplotlib.org/stable/api/animation_api.html)*

In [31]:
#| echo: false
#| output: asis
show_doc(f_create_polar_sonar_animation)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L2946){target="_blank" style="float:right; font-size:smaller"}

### f_create_polar_sonar_animation

>      f_create_polar_sonar_animation
>                                      (m_sonarReceivedIntensity_micron_final:nu
>                                      mpy.ndarray,
>                                      v_angles_rad_micron_final:numpy.ndarray)

*Create an animation of sonar data in polar view, visualizing the scanning process over time.

::: {.callout-note}
This function generates a dynamic polar plot of sonar data, simulating the rotating scan 
of a sonar sensor. It's particularly useful for visualizing how the sonar beam sweeps 
across the environment, providing an intuitive representation of the scanning process 
and detected features.
:::

**Parameters**  
 \ \  - `m_sonarReceivedIntensity_micron_final`: numpy.ndarray of sonar intensity data, shape (n_ranges, n_angles)  
 \ \  - `v_angles_rad_micron_final`: numpy.ndarray of angle data in radians, shape (n_angles,)  

**Returns**  
 \ \  matplotlib.animation.FuncAnimation: Animation object of the polar sonar scan  

**Example**

```python
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# Generate sample sonar data
d_n_ranges = 397  # 397 bins over 20 meters
d_n_angles = 2000  # Multiple revolutions
m_sonarReceivedIntensity_micron_final = np.random.rand(d_n_ranges, d_n_angles)
v_angles_rad_micron_final = np.linspace(0, 10*2*np.pi, d_n_angles)  # 10 full rotations

# Create some artificial features in the sonar data
for i in range(10):  # Create features in each rotation
    angle_start = i * 200
    m_sonarReceivedIntensity_micron_final[50:70, angle_start+30:angle_start+50] = 0.8  # Object at about 45 degrees
    m_sonarReceivedIntensity_micron_final[100:120, angle_start+90:angle_start+110] = 0.9  # Object at about 180 degrees
    m_sonarReceivedIntensity_micron_final[30:40, angle_start+150:angle_start+170] = 0.7  # Object at about 270 degrees

# Create animation
anim = f_create_polar_sonar_animation(m_sonarReceivedIntensity_micron_final, v_angles_rad_micron_final)

# Display animation in notebook
HTML(anim.to_jshtml())
```

::: {.callout-tip collapse=true}
#### **Details on the polar sonar animation process**

1. **Animation Setup**:
   - Creates a figure with a polar projection, suitable for circular sonar scans.
   - Initializes the plot with the first few data points.

2. **Polar Plot Customization**:
   - Sets the angle direction to clockwise with 0 degrees at the top, matching typical sonar conventions.
   - Customizes angular tick labels to show degrees from 0° to 330°.

3. **Color Mapping**:
   - Uses a colorbar to represent sonar intensity, allowing for easy interpretation of signal strength.

4. **Animation Function**:
   - Defines `f_animate` function that updates the plot for each frame.
   - Adds new data to the plot in small increments, simulating a rotating sonar beam.
   - Handles wraparound when the scan completes a full revolution.

5. **Animation Creation**:
   - Uses `matplotlib.animation.FuncAnimation` to create the animation.
   - Sets the frame range, interval, and links it to the update function.

This approach provides a dynamic visualization of the sonar scanning process, allowing 
viewers to observe how the sonar data is built up over time and how features in the 
environment appear and evolve during the scan.
:::

**References**  
\ \ 1. [Matplotlib Animation in Polar Coordinates](https://matplotlib.org/stable/gallery/animation/animate_polar.html)*

In [32]:
#| echo: false
#| output: asis
show_doc(f_micron_with_trajectory)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L3071){target="_blank" style="float:right; font-size:smaller"}

### f_micron_with_trajectory

>      f_micron_with_trajectory (m_interp_pos_micron_final:numpy.ndarray,
>                                m_sonarReceivedIntensity_micron_final:numpy.nda
>                                rray, v_range_micron:numpy.ndarray)

*Create an interactive visualization of AUV trajectory and sonar data.

::: {.callout-note}
This function generates an interactive plot combining the AUV's 3D trajectory with its 
horizontal sonar data. It allows users to explore different segments of the mission 
using a slider, providing a comprehensive view of the AUV's movement and the 
underwater environment it's scanning.
:::

**Parameters**  
 \ \  - `m_interp_pos_micron_final`: numpy.ndarray of interpolated AUV position data, shape (n_points, 3)  
 \ \  - `m_sonarReceivedIntensity_micron_final`: numpy.ndarray of sonar intensity data, shape (n_ranges, n_points)  
 \ \  - `v_range_micron`: numpy.ndarray representing the range values of the sonar data  

**Returns**  
 \ \  ipywidgets.interactive: Interactive widget for exploring the AUV trajectory and sonar data  

**Example**

```python
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
from IPython.display import display

# Generate sample data
d_n_points = 50000
d_n_ranges = 397

# AUV trajectory data
m_interp_pos_micron_final = np.column_stack((
    np.linspace(0, 100, d_n_points),  # x
    10 * np.sin(np.linspace(0, 8*np.pi, d_n_points)),  # y
    -5 * np.cos(np.linspace(0, 4*np.pi, d_n_points))   # z
))

# Sonar intensity data
m_sonarReceivedIntensity_micron_final = np.random.rand(d_n_ranges, d_n_points)
v_range_micron = np.linspace(0, 20, d_n_ranges)

# Create interactive widget
widget = f_micron_with_trajectory(m_interp_pos_micron_final, 
                                  m_sonarReceivedIntensity_micron_final, 
                                  v_range_micron)

# Display the widget
display(widget)
```

::: {.callout-tip collapse=true}
#### **Details on the interactive visualization process**

1. **Image Loading**:
   - Loads and resizes a background image (e.g., satellite view of the mission area) for context.

2. **Update Function**:
   - Defines `update` function that is called whenever the slider value changes.
   - Creates a new figure for each update with a complex grid layout.

3. **Trajectory Visualization**:
   - Plots the full AUV trajectory and the current segment in two 3D subplots.
   - One subplot shows a top-down view, the other a perspective view.
   - Uses color-coding for start and end points, and different colors for full and current trajectories.

4. **Sonar Data Visualization**:
   - Displays the horizontal sonar data as a 2D color plot below the trajectory plots.
   - Shows a segment of sonar data corresponding to the current position in the trajectory.

5. **Layout and Styling**:
   - Uses a grid layout to organize the background image, 3D trajectory plots, and sonar data plot.
   - Applies consistent styling, including font sizes and color schemes.

6. **Interactive Widget Creation**:
   - Creates an interactive slider widget for selecting different segments of the mission data.
   - Links the slider to the update function for dynamic visualization.

This approach provides an intuitive and interactive way to explore the relationship between 
the AUV's movement through 3D space and the sonar data it collects, enabling detailed 
analysis of the mission data and environmental features.
:::

**References**  
\ \ 1. [ipywidgets Documentation](https://ipywidgets.readthedocs.io/en/latest/)*

In [33]:
#| echo: false
#| output: asis
show_doc(f_seaking_with_trajectory)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L3263){target="_blank" style="float:right; font-size:smaller"}

### f_seaking_with_trajectory

>      f_seaking_with_trajectory (m_interp_pos_micron_final:numpy.ndarray,
>                                 m_sonarReceivedIntensity_micron_final:numpy.nd
>                                 array, v_range_micron:numpy.ndarray)

*Create an interactive visualization of AUV trajectory and vertical sonar data.

::: {.callout-note}
This function generates an interactive plot combining the AUV's 3D trajectory with its 
vertical sonar data. It allows users to explore different segments of the mission 
using a slider, providing a comprehensive view of the AUV's movement and the 
vertical profile of the underwater environment it's scanning.
:::

**Parameters**  
 \ \  - `m_interp_pos_micron_final`: numpy.ndarray of interpolated AUV position data, shape (n_points, 3)  
 \ \  - `m_sonarReceivedIntensity_micron_final`: numpy.ndarray of vertical sonar intensity data, shape (n_ranges, n_points)  
 \ \  - `v_range_micron`: numpy.ndarray representing the range values of the sonar data  

**Returns**  
 \ \  ipywidgets.interactive: Interactive widget for exploring the AUV trajectory and vertical sonar data  

**Example**

```python
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
from IPython.display import display

# Generate sample data
d_n_points = 100000
d_n_ranges = 50

# AUV trajectory data
m_interp_pos_micron_final = np.column_stack((
    np.linspace(0, 100, d_n_points),  # x
    10 * np.sin(np.linspace(0, 8*np.pi, d_n_points)),  # y
    -5 * np.cos(np.linspace(0, 4*np.pi, d_n_points))   # z
))

# Vertical sonar intensity data
m_sonarReceivedIntensity_micron_final = np.random.rand(d_n_ranges, d_n_points)
v_range_micron = np.linspace(0, 10, d_n_ranges)

# Create interactive widget
widget = f_seaking_with_trajectory(m_interp_pos_micron_final, 
                                   m_sonarReceivedIntensity_micron_final, 
                                   v_range_micron)

# Display the widget
display(widget)
```

::: {.callout-tip collapse=true}
#### **Details on the interactive visualization process**

1. **Image Loading**:
   - Loads and resizes a background image (e.g., satellite view of the mission area) for context.

2. **Update Function**:
   - Defines `update` function that is called whenever the slider value changes.
   - Creates a new figure for each update with a complex grid layout.

3. **Trajectory Visualization**:
   - Plots the full AUV trajectory and the current segment in two 3D subplots.
   - One subplot shows a top-down view, the other a perspective view.
   - Uses color-coding for start and end points, and different colors for full and current trajectories.

4. **Vertical Sonar Data Visualization**:
   - Displays the vertical sonar data as a 2D color plot below the trajectory plots.
   - Shows a segment of sonar data corresponding to the current position in the trajectory.
   - The y-axis represents depth (50 bins over 10 meters), providing a vertical profile view.

5. **Layout and Styling**:
   - Uses a grid layout to organize the background image, 3D trajectory plots, and sonar data plot.
   - Applies consistent styling, including font sizes and color schemes.

6. **Interactive Widget Creation**:
   - Creates an interactive slider widget for selecting different segments of the mission data.
   - Links the slider to the update function for dynamic visualization.

This approach provides an intuitive and interactive way to explore the relationship between 
the AUV's movement through 3D space and the vertical sonar data it collects, enabling detailed 
analysis of the mission data, bathymetry, and vertical environmental features.
:::

**References**  
\ \ 1. [ipywidgets Documentation](https://ipywidgets.readthedocs.io/en/latest/)*

## Utility Functions
> General-purpose utility functions and configuration settings.

In [34]:
#| echo: false
#| output: asis
show_doc(f_format_time)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L3456){target="_blank" style="float:right; font-size:smaller"}

### f_format_time

>      f_format_time (d_timestamp:float)

*Convert timestamp to HH:MM:SS format.

::: {.callout-note}
This function converts a Unix timestamp to a formatted time string using UTC.
:::

**Parameters**  
 \ \  - `d_timestamp`: Unix timestamp (seconds since epoch)  

**Returns**  
 \ \  str: Formatted time string in HH:MM:SS format  

**Example**

```python
d_timestamp = 1609459200  # 2021-01-01 00:00:00 UTC
s_formatted_time = f_format_time(d_timestamp)
print(s_formatted_time)  # Output: "00:00:00"
```*

In [35]:
#| echo: false
#| output: asis
show_doc(f_add_packages_to_all)

---

[source](https://github.com/20KUTS/nautilopy/blob/main/nautilopy/core.py#L3481){target="_blank" style="float:right; font-size:smaller"}

### f_add_packages_to_all

>      f_add_packages_to_all ()

*Add commonly used package names to the global __all__ list.

::: {.callout-note}
This function populates or extends the global __all__ list with names of commonly uszed 
packages and modules. This is useful for controlling what is exported when using 
'from module import *' in other parts of the project.
:::

**Parameters**  
 \ \  None

**Returns**  
 \ \  None

**Example**

```python
# Before calling the function
print(globals().get('__all__', 'Not defined'))

# Call the function
f_add_packages_to_all()

# After calling the function
print(__all__)
```

::: {.callout-tip collapse=true}
#### **Details on package addition process**

1. **Global __all__ Creation**:
   - Checks if __all__ exists in the global namespace.
   - If not, it creates an empty __all__ list.

2. **Package List**:
   - Defines a comprehensive list of commonly used package and module names.
   - Includes packages for numerical computing, data manipulation, visualization, 
     scientific computing, and more.

3. **List Extension**:
   - Extends the __all__ list with the defined package names.

4. **Usage in Modules**:
   - This function is typically called at the end of a module file.
   - It ensures that all necessary package names are included in __all__.

5. **Import Control**:
   - Helps in controlling what is imported when using 'from module import *'.
   - Useful for creating a clean namespace when importing the module.

6. **Maintenance**:
   - The list of packages can be easily updated or extended as project requirements change.

Note: While this approach can be convenient, it's generally recommended to use 
explicit imports in production code for better readability and maintainability.
:::

**References**  
\ \ 1. [Python __all__ special variable](https://docs.python.org/3/tutorial/modules.html#importing-from-a-package)  
\ \ 2. [PEP 8 -- Style Guide for Python Code](https://www.python.org/dev/peps/pep-0008/#imports)  
\ \ 3. [Python Modules and Packages](https://docs.python.org/3/tutorial/modules.html)*